# [5] New Conversation Flow

Internal documents indicate that this recently acquired company was performing many A/B tests; we would like to investigate one. This platform had a conversation page where owners and service providers could exchange messages as they organized their booking. The team thought this page could use a re-design and set out to improve its UI. A product manager then set up a test to measure the new page's effectiveness. On 2017-03-13 , an A/B test was launched. For those owners who sent a request, they would be randomly assigned to variant or holdout groups. Those users who are in the variant group would see the new conversation flow. However, those in the holdout group would see the old conversation flow. Providers would always see the old conversation flow.

In [1]:
cd ..

/home/jovyan/Business_Analytics/Business-Insights-and-Analytics/Marketing_Analytics


In [2]:
%run __init__.py
%matplotlib inline

In [13]:
conn = sqlite3.connect('db26.sqlite3')

## 1. Did conversations with the new conversation page book at a higher rate? - Yes

In [14]:
query_test = ('SELECT p.test_group, COUNT(p.test_name) AS total_number_people, COUNT(c.booked_at) AS booked_at \
               FROM conversations_conversation c \
               JOIN people_testsegmentation p \
               ON (c.requester_id = p.person_id) \
               WHERE p.test_name = "New Conversation Flow" \
               GROUP BY p.test_group;')

In [15]:
test_results = pd.read_sql_query(query_test, conn)
test_results

,test_group,total_number_people,booked_at
0,holdout,22895,9174
1,variant,22941,10136


In [16]:
test_results['booking_rate'] = (test_results.booked_at / test_results.total_number_people) * 100
test_results

,test_group,total_number_people,booked_at,booking_rate
0,holdout,22895,9174,40.069884
1,variant,22941,10136,44.182904


In [17]:
test_results.rename(columns={
    'test_group': 'Test Group Name',
    'total_number_people': 'Total Number of People',
    'booked_at': 'Number of Bookings',
    'booking_rate': 'Booking Rate (%)'
}, inplace=True)

In [18]:
test_results

,Test Group Name,Total Number of People,Number of Bookings,Booking Rate (%)
0,holdout,22895,9174,40.069884
1,variant,22941,10136,44.182904


In [33]:
((9174 + 10136) / (22895 + 22941))

0.4212845798062658

## 2. Is it statistically significant?

$H_0$: The booking rate for the variant group was not different than the booking rate of the holdout group.  

$H_1$: The booking rate for the variant group was different than the booking rate of the holdout group.

In [19]:
query_new_conv = ('SELECT p.test_group, p.test_name, c.booked_at \
                   FROM conversations_conversation c \
                   JOIN people_testsegmentation p \
                   ON (c.requester_id = p.person_id) \
                   WHERE p.test_name = "New Conversation Flow";')

In [20]:
new_conv_flow = pd.read_sql_query(query_new_conv, conn)
new_conv_flow.head(2)

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,2017-05-26 21:02:09.499404
1,holdout,New Conversation Flow,None


In [21]:
new_conv_flow = new_conv_flow.fillna(0)
new_conv_flow.head(2)

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,2017-05-26 21:02:09.499404
1,holdout,New Conversation Flow,0


In [22]:
new_conv_flow['booked_at'] = new_conv_flow.booked_at.apply(lambda x = 1: x != 0)
new_conv_flow.head(2)

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,True
1,holdout,New Conversation Flow,False


In [23]:
# number 1 means conversation booked
new_conv_flow.booked_at = new_conv_flow.booked_at*1
new_conv_flow.head(2)

,test_group,test_name,booked_at
0,holdout,New Conversation Flow,1
1,holdout,New Conversation Flow,0


In [24]:
holdout_mask     = new_conv_flow[new_conv_flow["test_group"] == 'holdout']["booked_at"]
variant_mask     = new_conv_flow[new_conv_flow["test_group"] == 'variant']["booked_at"]

In [25]:
ttest4 = stats.ttest_ind(holdout_mask, variant_mask)
ttest4

Ttest_indResult(statistic=-8.9244437702831156, pvalue=4.6409650880993761e-19)

In [26]:
new_conv_flow['test_group_hold_bool'] = new_conv_flow.test_group == 'holdout'

new_conv_flow.test_group_hold_bool = new_conv_flow.test_group_hold_bool*1
new_conv_flow.head(2)

,test_group,test_name,booked_at,test_group_hold_bool
0,holdout,New Conversation Flow,1,1
1,holdout,New Conversation Flow,0,1


In [27]:
new_holdout_mask     = new_conv_flow[new_conv_flow["test_group_hold_bool"] == 1]["booked_at"]
new_variant_mask     = new_conv_flow[new_conv_flow["test_group_hold_bool"] == 0]["booked_at"]

In [38]:
holdout_mask.mean()

0.40069884254203975

In [39]:
variant_mask.mean()

0.44182903971056187

`ttest_ind` tests 2 independent samples to see if they have different means.

In [28]:
ttest3 = stats.ttest_ind(new_variant_mask, new_holdout_mask)
ttest3

Ttest_indResult(statistic=8.9244437702831156, pvalue=4.6409650880993761e-19)

## Analysis:
- The calculated pvalue is 4.6409650880993761e-19, which is smaller than 0.05 so I conclude that the difference is not significant.

## 3. Do you have any reservations about the experiment design? What would you recommend as next steps?

### Experiment Design:
- I know that owners were randomly selected to be in either group, what about testing providers, does that change booking rate at all?
- Find out male/female ratios, any demographic information, ages of users in each sample, ethnicity, location. Are percentages of each sample representative of the universe.
- How often are the people selected using the service? Is it their first time, what percentage are regular users, does that number reflect average percentage of regular users.
- How accessible is the new interface compared with the old interface, is there reason to believe it is more/less accessible.
- Was any of the data weighted to ensure the correct split (eg 50/50 between m/f)
- Is the data normally distributed?
